## Load and clean the data

In [4]:
# In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from
# https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
# In this dataset our desired target for classification task will be converted variable - has the client signed up to the platform or not.

# Data preparation
# Check if the missing values are presented in the features.
# If there are missing values:
# For caterogiral features, replace them with 'NA'
# For numerical features, replace with with 0.0
# Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. 
# Use train_test_split function for that with random_state=1

# run once
# !wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv